In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as spa

import sys; sys.path.insert(0, '..')

import hylife.utilitis_FEEC.basics.spline_evaluation_3d as eva_3d

import hylife.utilitis_FEEC.bsplines     as bsp 
import hylife.utilitis_FEEC.Bspline      as bspline
import hylife.utilitis_FEEC.spline_space as spl

import hylife.utilitis_PIC.STRUPHY_fields     as pic
import hylife.utilitis_PIC.STRUPHY_fields_new as pic_new

In [8]:
# setup
Nel        = [8, 6, 10]                    # number of elements on logical domain
bc         = [False, False, False]         # boundary conditions (True: periodic, False: else)
p          = [3, 2, 1]                     # spline degrees  

el_b       = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                       # element boundaries
T          = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]   # knot vectors (for N functions)

nq_el      = [6, 6, 6]                     # number of quadrature points per element for integrations over whole domain

spaces     = [spl.spline_space_1d(T, p, bc, nq_el) for T, p, bc, nq_el in zip(T, p, bc, nq_el)] # 1d B-spline spaces
t_space    = spl.tensor_spline_space(spaces)

N          = [bspline.Bspline(spl.T, spl.p)     for spl in spaces]     # N-splines for comparison
D          = [bspline.Bspline(spl.t, spl.p - 1) for spl in spaces]     # D-splines for comparison

kind_map   = 1
params_map = [1., 2., 3.]

Np = 10 # number of particles

# spline coefficients
u1 = np.random.rand(t_space.NbaseD[0], t_space.NbaseN[1], t_space.NbaseN[2])
u2 = np.random.rand(t_space.NbaseN[0], t_space.NbaseD[1], t_space.NbaseN[2])
u3 = np.random.rand(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseD[2])

# particles
particles_pos = np.random.rand(Np, 3)

u_part     = np.empty((Np, 3), dtype=float, order='F')
u_part_new = np.empty((Np, 3), dtype=float)

In [12]:
# ==== coefficients for pp-forms in interval [0, delta] (N and D) for fast field evaluation
pp0 = []
pp1 = []

delta = t_space.delta

for i in range(3):
    
    if   p[i] == 1:
        pp0.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]]))
        pp1.append(np.asfortranarray([[1/delta[i]]]))
    
    elif p[i] == 2:
        pp0.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]]))
        pp1.append(np.asfortranarray([[1., -1/delta[i]], [0., 1/delta[i]]])/delta[i])
    
    elif p[i] == 3:
        pp0.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]]))
        pp1.append(np.asfortranarray([[1/2, -1/delta[i], 1/(2*delta[i]**2)], [1/2, 1/delta[i], -1/delta[i]**2], [0., 0., 1/(2*delta[i]**2)]])/delta[i])
    
    elif p[i] == 4:
        pp0.append(np.asfortranarray([[1/24, -1/(6*delta[i]), 1/(4*delta[i]**2), -1/(6*delta[i]**3), 1/(24*delta[i]**4)], [11/24, -1/(2*delta[i]), -1/(4*delta[i]**2), 1/(2*delta[i]**3), -1/(6*delta[i]**4)], [11/24, 1/(2*delta[i]), -1/(4*delta[i]**2), -1/(2*delta[i]**3), 1/(4*delta[i]**4)], [1/24, 1/(6*delta[i]), 1/(4*delta[i]**2), 1/(6*delta[i]**3), -1/(6*delta[i]**4)], [0., 0., 0., 0., 1/(24*delta[i]**4)]]))
        pp1.append(np.asfortranarray([[1/6, -1/(2*delta[i]), 1/(2*delta[i]**2), -1/(6*delta[i]**3)], [2/3, 0., -1/delta[i]**2, 1/(2*delta[i]**3)], [1/6, 1/(2*delta[i]), 1/(2*delta[i]**2), -1/(2*delta[i]**3)], [0., 0., 0., 1/(6*delta[i]**3)]])/delta[i])
    
    else:
        print('Spline degree > 4 not yet implemented!')
# ======================================================================


# number of basis functions in different spaces
NbaseN = t_space.NbaseN
NbaseD = t_space.NbaseD

Nbase_0form =  [NbaseN[0], NbaseN[1], NbaseN[2]]
Nbase_1form = [[NbaseD[0], NbaseN[1], NbaseN[2]], [NbaseN[0], NbaseD[1], NbaseN[2]], [NbaseN[0], NbaseN[1], NbaseD[2]]]
Nbase_2form = [[NbaseN[0], NbaseD[1], NbaseD[2]], [NbaseD[0], NbaseN[1], NbaseD[2]], [NbaseD[0], NbaseD[1], NbaseN[2]]]
Nbase_3form =  [NbaseD[0], NbaseD[1], NbaseD[2]]

Ntot_0form  =  NbaseN[0]*NbaseN[1]*NbaseN[2] 
Ntot_1form  = [NbaseD[0]*NbaseN[1]*NbaseN[2], NbaseN[0]*NbaseD[1]*NbaseN[2], NbaseN[0]*NbaseN[1]*NbaseD[2]]
Ntot_2form  = [NbaseN[0]*NbaseN[1]*NbaseD[2], NbaseD[0]*NbaseN[1]*NbaseD[2], NbaseD[0]*NbaseD[1]*NbaseN[2]]  
Ntot_3form  =  NbaseD[0]*NbaseD[1]*NbaseD[2]

In [18]:
# evaluation
pic.evaluate_1form(np.asfortranarray(particles_pos), T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], p, Nel, np.asfortranarray(Nbase_1form), Np, np.asfortranarray(u1), np.asfortranarray(u2), np.asfortranarray(u3), pp0[0], pp0[1], pp0[2], pp1[0], pp1[1], pp1[2], u_part, kind_map, params_map)

In [19]:
u_part

array([[5.12877929, 1.46368919, 6.38369047],
       [7.68244978, 4.34873931, 3.10740353],
       [4.73344766, 3.7101721 , 3.53480218],
       [2.13024362, 3.70943206, 5.24399595],
       [4.92983706, 2.20029175, 4.51769784],
       [2.94429288, 2.7227373 , 4.69080843],
       [2.1269124 , 1.98720977, 4.91986886],
       [3.54247841, 3.34623939, 6.97024273],
       [4.17031066, 3.41818404, 2.82295915],
       [3.51362752, 2.70885535, 3.2813366 ]])

In [25]:
pic_new.evaluate_1form(particles_pos, T[0], T[1], T[2], t_space.t[0], t_space.t[1], t_space.t[2], p, Nel, t_space.NbaseN, t_space.NbaseD, Np, u1, u2, u3, u_part_new, kind_map, params_map)

In [26]:
u_part_new

array([[5.12877929, 1.46368919, 6.38369047],
       [7.68244978, 4.34873931, 3.10740353],
       [4.73344766, 3.7101721 , 3.53480218],
       [2.13024362, 3.70943206, 5.24399595],
       [4.92983706, 2.20029175, 4.51769784],
       [2.94429288, 2.7227373 , 4.69080843],
       [2.1269124 , 1.98720977, 4.91986886],
       [3.54247841, 3.34623939, 6.97024273],
       [4.17031066, 3.41818404, 2.82295915],
       [3.51362752, 2.70885535, 3.2813366 ]])